In [ ]:
import os
import os.path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from evaluation_base import linearRegression
from IPython.display import display
import ipywidgets as widgets

plt.rcParams['figure.figsize'] = 10, 6


def H1(text):
    return widgets.HTML(f"<h1>{text}</h1>")
    
def H2(text):
    return widgets.HTML(f"<h2>{text}</h2>")

def c_s(T):
    return 331 * np.sqrt(T / 273)

def c(lamb, f=2e3):
    return lamb * f

def evaluate_all(data_dir):
    display(H1("Schallgeschwindigkeit durch Temperaturessung"))
    display(H2("Teil 1: Quinck pipe"))
    
    minima = np.load(os.path.join(data_dir, "DataV8Part1.npy"))
    diffs1 = np.diff(minima)
    print("Differenzen [cm]:", diffs1)
    lamb1 = np.mean(diffs1) * 1e-2 * 2
    print("Wellenlaenge = {:.5f} m".format(round(lamb1, 5)))
    c1 = c(lamb1, f=2e3)
    print("Schallgeschwindigkeit = {0} m/s".format(round(c1, 1)))
    
    T = 273 + 20
    cs = c_s(T)
    print("Schallgeschwindigkeit (erwartet) = {0} m/s".format(round(cs, 1)))

    display(H2("Teil 2: Ultraschall"))
        
    distances = np.load(os.path.join(data_dir, "DataV8Part2.npy"))
    diffs2 = np.diff(distances)
    print("Differenzen [cm]:", diffs2)
    lamb2 = np.mean(diffs2) * 1e-3
    print("Wellenlaenge = {0} m".format(round(lamb2, 4)))
    c2 = c(lamb2, f=40692)
    print("Schallgeschwindigkeit = {0} m/s".format(round(c2, 1)))

    display(H2("Teil 3: Echolot Verfahren"))
    
    data3 = np.load(os.path.join(data_dir, "DataV8Part3.npy"))
    distances = data3[0]
    times = data3[1]

    linearRegression(times, distances, "Zeiten [ms]", "Distanzen [mm]")
    slope, intercept = linearRegression(distances, times, "Distanzen [mm]", "Zeiten [ms]")
    # 2 because it is the time for back and forth, but distance is one way
    c3 = round(2 / slope, 1)
    print("Schallgeschwindigkeit = {0} m/s".format(c3))
    
    display(H2("Fragen nach der Durchfuehrung"))
    
    # Orgelpfeifenlänge:
    lamb_orgel = 340 / 440 # m
    # orgellänge = lamb / 4
    l_orgel = lamb_orgel / 4
    print("Länge der Orgelpfeife : %.3f m" % l_orgel)
    # Wellenlänge Ultraschallfrq:
    lamb_ultra = 1600 / 8e6 * 1e6
    print("Ultraschallwellenlänge : %.0f \u03BCm" % lamb_ultra)

# Interactive student selection
with open("students.txt", "r") as f:
    students = [line.strip("\n") for line in f if not line.startswith("#")]
    
tb = widgets.ToggleButtons(
    options=students,
    description='Name:',
)
display(tb)

output = widgets.Output()
display(output)

@output.capture(clear_output=True)
def on_student_change(change):
    name = change["new"]
    display(H1(f"Auswertung für: {name}"))

    name_escaped = name.replace(" ", "_")
    data_dir = os.path.join("V8", name_escaped)
    
    evaluate_all(data_dir)

tb.observe(on_student_change, names="value")
on_student_change({"new": students[0]})